In [1]:
import pyscal.crystal_structures as pcs
import pyscal.csystem as pcm
import pyscal.core as pc
from ase import atoms
from ase.io import read, write
import numpy as np
import matplotlib.pyplot as plt

In [2]:
atoms, box = pcs.make_crystal(structure='bcc', 
                              lattice_constant=3.127, repetitions=(2,2,2),)

In [3]:
sys = pc.System()
sys.box = box
sys.atoms = atoms

In [4]:
sys.natoms

16

In [6]:
sys.atoms.keys()

dict_keys(['positions', 'ids', 'types', 'ghost'])

In [7]:
pcm.reset_all_neighbors(sys.atoms)

In [9]:
box = [[156.35,   0.  ,   0.  ],
 [  0.  , 156.35,   0.  ],
 [  0.  ,   0.  , 156.35]]
boxd = [156.35,156.35,156.35]

In [10]:
pcm.get_all_neighbors_normal(sys.atoms, 3.6, 0,
                           0, sys.rot, sys.rotinv, sys.boxdims)

In [11]:
atoms['neighbors']

[[1, 2, 4, 8, 527, 541, 603, 616, 617, 919, 932, 933, 994, 995],
 [0, 2, 3, 4, 5, 6, 8, 9, 10, 12, 14, 617, 933, 995],
 [0, 1, 3, 6, 10, 541, 543, 617, 618, 619, 933, 934, 935, 1008],
 [1, 2, 6, 7, 10, 11, 14, 619, 935, 1008, 1009, 1012, 1016, 1020],
 [0, 1, 5, 6, 12, 603, 607, 617, 620, 621, 995, 998, 999, 1072],
 [1, 4, 6, 7, 12, 13, 14, 621, 999, 1072, 1073, 1074, 1080, 1082],
 [1, 2, 3, 4, 5, 7, 14, 617, 619, 621, 622, 623, 1012, 1074],
 [3, 5, 6, 14, 15, 623, 1012, 1013, 1020, 1074, 1075, 1082, 1088, 1096],
 [0, 1, 9, 10, 12, 919, 927, 933, 940, 941, 995, 1002, 1003, 1392],
 [1, 8, 10, 11, 12, 13, 14, 941, 1003, 1392, 1393, 1394, 1396, 1398],
 [1, 2, 3, 8, 9, 11, 14, 933, 935, 941, 942, 943, 1016, 1394],
 [3, 9, 10, 14, 15, 943, 1016, 1017, 1020, 1394, 1395, 1398, 1408, 1412],
 [1, 4, 5, 8, 9, 13, 14, 995, 999, 1003, 1006, 1007, 1080, 1396],
 [5, 9, 12, 14, 15, 1007, 1080, 1081, 1082, 1396, 1397, 1398, 1472, 1474],
 [1, 3, 5, 6, 7, 9, 10, 11, 12, 13, 15, 1020, 1082, 1398],
 [7, 11

In [7]:
[box[0][0],box[1][1],box[2][2]]

[156.34999999999997, 156.34999999999997, 156.34999999999997]

In [ ]:
atoms

In [39]:
positions = np.array(atoms['positions'])
del atoms['positions']

In [40]:
positions[-1]

array([29.7065, 29.7065, 29.7065])

In [14]:
ps = []
for pos in positions:
    ps.append(len(pos))

In [22]:
ps[-1]

248000

In [30]:
positions = np.array([np.array(pos) for pos in positions], dtype=np.float64)

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (2001,) + inhomogeneous part.

In [41]:
pcm.get_all_neighbors_normal(positions, atoms, 3.6, 0, 0, box, box, boxd)

TypeError: 'numpy.ndarray' object is not callable

In [11]:
atomdict.keys()

dict_keys(['positions', 'neighbors', 'neighbordist', 'neighborweight', 'diff', 'r', 'phi', 'theta', 'cutoff'])

In [12]:
atomdict['theta'][0]

[2.186276035465284,
 3.141592653589793,
 0.0,
 0.95531661812451,
 1.5707963267948966,
 1.5707963267948966,
 2.1862760354652844,
 0.9553166181245096,
 1.5707963267948966,
 1.5707963267948966,
 2.1862760354652844,
 0.9553166181245096,
 2.186276035465285,
 0.9553166181245093]

In [2]:
atom = pca.Atom(pos=[1,1,1])

In [3]:
al = pcm.AtomVector()

In [6]:
al.emplace_back(atom)

AttributeError: 'pyscal.cmodsystem.AtomVector' object has no attribute 'emplace_back'

In [6]:
atom.neighbors

[]

In [7]:
al = [atom]

In [8]:
pcm.perturb_atom(al)

TypeError: perturb_atom(): incompatible function arguments. The following argument types are supported:
    1. (arg0: std::vector<Atom, std::allocator<Atom> >) -> None

Invoked with: [<pyscal.catom.Atom object at 0x7f634c422470>]

Did you forget to `#include <pybind11/stl.h>`? Or <pybind11/complex.h>,
<pybind11/functional.h>, <pybind11/chrono.h>, etc. Some automatic
conversions are optional and require extra headers to be included
when compiling your pybind11 module.

In [7]:
al[0].neighbors

[]

In [11]:
atoms[0].coordination

304